利用chatgpt API產生fine-tuning訓練資料

In [1]:
from openai import OpenAI

client = OpenAI(api_key="<api key>")

In [ ]:
import json
with open("./json/taoyuan_airport.json", 'rb') as infile:
    airport_qa = json.load(infile)

In [33]:
airport_qa

{'faqs': [{'question': '請問動物申請轉機需要之文件',
   'answer': '1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。\n2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。'},
  {'question': '護照快過期是否會影響轉機',
   'answer': '一般而言，各國皆規定護照效期需6個月以上才可入境，如護照效期不足雖可轉機，但將無法入境目的國。'},
  {'question': '請問因轉機，可否在管制區內過夜',
   'answer': '轉機旅客可過夜休息區域：\n1. 第二航廈轉機休息區：於第二航廈管制區3樓出發層D區設有13張躺椅供旅客休息。\n2. 第一、二航廈管制區4樓出發層設有環亞付費貴賓室。\n3. 第一航廈管制區4樓出發層環亞貴賓室旁設有免費休憩空間。\n4. 第二航廈管制區4樓出發層北側設有免費休憩空間。'},
  {'question': '請問轉機需要預留多少時間才夠',
   'answer': '考量跨航廈轉機移動距離，建議應預留開始辦理登機時間前40～60分鐘時間。'},
  {'question': '請問轉機是否要通過證照查驗、海關、安檢',
   'answer': '轉機屬於過境，因為沒有實際入境，原則不需經過海關申報、入境證照查驗，僅須就隨身物品再經過一次轉機安檢即可；但轉機旅客若有入境再出境之需要(如辦理票務作業、重新託運行李等)，則須通過證照查驗、海關、安檢等通關流程。'},
  {'question': '請問轉機，但不入境臺灣是否需要簽證', 'answer': '無論本國籍或外國籍，皆不用簽證。'},
  {'question': '請問不同航廈轉機要多少時間，動線如何進行',
   'answer': '1. 桃園機場登機門分為A1-A9、B1-B9&amp;B1R、C1-C10&amp;C5R、D1-D10&amp;D5R，A、B、C、D區共設有4個轉機點，其中A、B區位於第一航廈，C、D區位於第二航廈。\n2. 不同航廈轉機可透過步行或電車（Skytrain），旅客可依據下段航程之登機門資訊，循轉機標誌步行或搭乘電車前往，進而抵達正確之轉機點。\n3. 桃園機場抵

- 先將同一個問題利用chatgpt API產生10句意思相同但敘述不同之文字
- 再將問題對應到相同的解答
- 最後產生fine-tuning需要的格式

In [20]:
def create_10_same_question(question):
    alt="""
        [
            "動物申請轉機需要的文件是什麼？",
            "請問哪些文件是動物申請轉機需要的？",
            "為了動物申請轉機，需要提交哪些文件？",
            "請問動物轉機申請需要哪些文件？",
            "動物轉機所需的文件有哪些？",
            "想請問動物轉機申請時需要的文件有哪些？",
            "請問申請動物轉機需要哪些檔案？",
            "對於動物轉機申請，應該提供什麼文件？",
            "動物轉機所需的文件是什麼？"
        ]
    """
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "請寫出9個與下面文句意思相同但敘述不同的句子，以python array表示，並將原始文句當作array第一個元素"},
            {"role": "assistant", "content": alt},
            {"role": "user", "content": question}
        ]
    )
    # return completion['choices'][0]['message']['content']
    return completion.choices[0].message.content


In [35]:
def create_traing_datasets(question, answer):
    ques_10 = create_10_same_question(question)
    ques_list = json.loads(ques_10)
    traing_data=""
    for i in range(len(ques_list)):
        traing_data += f'{{"messages": [{{"role": "system", "content": "你是一個機場的智能客服。"}}, {{"role": "user", "content": "{ques_list[i]}"}}, {{"role": "assistant", "content": "{answer}"}}]}}\n'
    return traing_data

In [36]:
all_training_data = ""

for i in range(len(airport_qa['faqs'])):
    question = airport_qa['faqs'][i]['question']
    answer = airport_qa['faqs'][i]['answer'].replace('\n',"")
    data = create_traing_datasets(question, answer)
    all_training_data += data

with open('./json/taoyuan_airport_datasets.jsonl', 'w', encoding='utf-8') as outfile:
    outfile.write(all_training_data)

In [37]:
print(all_training_data)

{"messages": [{"role": "system", "content": "你是一個機場的智能客服。"}, {"role": "user", "content": "請問動物申請轉機需要哪些文件？"}, {"role": "assistant", "content": "1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。"}]}
{"messages": [{"role": "system", "content": "你是一個機場的智能客服。"}, {"role": "user", "content": "請問申請動物轉機時需要提交的文件有哪些？"}, {"role": "assistant", "content": "1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。"}]}
{"messages": [{"role": "system", "content": "你是一個機場的智能客服。"}, {"role": "user", "content": "煩請告知動物轉機申請所需文件清單。"}, {"role": "assistant", "content": "1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。"}]}
{"messages": [{"role": "system", "content": "你是一個機場的智能客服。"}, {"role": "user", "content": "請問動物轉機申請所需檔案是哪些？"}, {"role": "assistant", "content": "1. 攜帶活體動物出境需符合入境國相關檢疫規定，請先洽出境國檢疫單位確認完成相關檢疫申請程序。2. 有攜帶活體動物轉機需求時，亦需經本國檢疫署進行轉機檢查，建議可透過航空公司協助提出申請。"}]}
{"messages": [{"role": "